<a href="https://colab.research.google.com/github/tonyzamyatin/learning-pytorch/blob/master/fcc-course/01_workflow_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01. PyTorch Workflow Exercise Template

The following is a template for the PyTorch workflow exercises.

It's only starter code and it's your job to fill in the blanks.

Because of the flexibility of PyTorch, there may be more than one way to answer the question.

Don't worry about trying to be *right* just try writing code that suffices the question.

You can see one form of [solutions on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/extras/solutions) (but try the exercises below yourself first!).

In [2]:
# Import necessary libraries
import torch
from torch import nn
import matplotlib.pyplot as plt

In [38]:
# Setup device-agnostic code
device_type = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_type)

## 1. Create a straight line dataset using the linear regression formula (`weight * X + bias`).
  * Set `weight=0.3` and `bias=0.9` there should be at least 100 datapoints total.
  * Split the data into 80% training, 20% testing.
  * Plot the training and testing data so it becomes visual.

Your output of the below cell should look something like:
```
Number of X samples: 100
Number of y samples: 100
First 10 X & y samples:
X: tensor([0.0000, 0.0100, 0.0200, 0.0300, 0.0400, 0.0500, 0.0600, 0.0700, 0.0800,
        0.0900])
y: tensor([0.9000, 0.9030, 0.9060, 0.9090, 0.9120, 0.9150, 0.9180, 0.9210, 0.9240,
        0.9270])
```

Of course the numbers in `X` and `y` may be different but ideally they're created using the linear regression formula.

In [65]:
# Create the data parameters
weight = 0.3
bias = 0.9

# Make X and y using linear regression feature
X = torch.arange(start=0, end=1, step=0.01).unsqueeze(dim=1)
y = weight * X + bias

print(f"Shape of X: {X.shape}")
print(f"Number of X samples: {len(X)}")
print(f"Number of y samples: {len(y)}")
print(f"First 10 X & y samples:\nX: {X[:10]}\ny: {y[:10]}")

In [66]:
# Split the data into training and testing
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

len(X_train), len(y_train), len(X_test), len(y_test)

In [89]:
# Plot the training and testing data
def plot_predictions(train_data = X_train.cpu(),
                     train_labels = y_train.cpu(),
                     test_data = X_test.cpu(),
                     test_labels = y_test.cpu(),
                     predictions = None):
  plt.figure(figsize=(10, 7))

  plt.scatter(x=train_data, y=train_labels, c='b', s=4, label="Training data")
  plt.scatter(x=test_data, y=test_labels, c='g', s=4, label="Test data")

  if predictions is not None:
    plt.scatter(x=test_data, y=predictions, c='r', s=4, label="Predictions")

  plt.legend(prop={"size": 14})

plot_predictions()

## 2. Build a PyTorch model by subclassing `nn.Module`.
  * Inside should be a randomly initialized `nn.Parameter()` with `requires_grad=True`, one for `weights` and one for `bias`.
  * Implement the `forward()` method to compute the linear regression function you used to create the dataset in 1.
  * Once you've constructed the model, make an instance of it and check its `state_dict()`.
  * **Note:** If you'd like to use `nn.Linear()` instead of `nn.Parameter()` you can.

In [81]:
# Create PyTorch linear regression model by subclassing nn.Module
class LinearRegressionModel(nn.Module):

  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(in_features=1, out_features=1, bias=True)

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    return self.linear(x)

In [82]:
# Instantiate the model and put it to the target device
torch.manual_seed(42)
model = LinearRegressionModel()
model.to(device.type)

model.state_dict(), next(model.parameters()).device

## 3. Create a loss function and optimizer using `nn.L1Loss()` and `torch.optim.SGD(params, lr)` respectively.
  * Set the learning rate of the optimizer to be 0.01 and the parameters to optimize should be the model parameters from the model you created in 2.
  * Write a training loop to perform the appropriate training steps for 300 epochs.
  * The training loop should test the model on the test dataset every 20 epochs.

In [83]:
# Create the loss function and optimizer
learning_rate = 0.01
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=learning_rate)

In [88]:
# Training loop


# Train model for 300 epochs
epochs = 300

# Send data to target device
model.to(device.type)
X_train = X_train.to(device.type)
y_train = y_train.to(device.type)
X_test = X_test.to(device.type)
y_test = y_test.to(device.type)

for epoch in range(epochs):
  ### Training

  # Put model in train mode
  model.train()

  # 1. Forward pass
  pred_train = model(X_train)

  # 2. Calculate loss
  loss = loss_fn(pred_train, y_train)

  # 3. Zero gradients
  model.zero_grad()

  # 4. Backpropagation
  loss.backward()

  # 5. Step the optimizer
  optimizer.step()

  ### Perform testing every 20 epochs
  if epoch % 20 == 0:

    # Put model in evaluation mode and setup inference context
    model.eval()
    with torch.inference_mode():
      # 1. Forward pass
      pred_test = model(X_test)
      # 2. Calculate test loss
      test_loss = loss_fn(pred_test, y_test)
      # Print out what's happening
      print(f"Epoch: {epoch} | Train loss: {loss:.3f} | Test loss: {test_loss:.3f}")

print(f"Estimate paramters: {model.state_dict()}")
print(f"True parameters: weight={weight}, bias={bias}")

## 4. Make predictions with the trained model on the test data.
  * Visualize these predictions against the original training and testing data (**note:** you may need to make sure the predictions are *not* on the GPU if you want to use non-CUDA-enabled libraries such as matplotlib to plot).

In [93]:
# Make predictions with the model
model.eval()
with torch.inference_mode():
  test_pred = model(X_test)

In [94]:
# Plot the predictions (these may need to be on a specific device)
plot_predictions(predictions=test_pred.cpu())

## 5. Save your trained model's `state_dict()` to file.
  * Create a new instance of your model class you made in 2. and load in the `state_dict()` you just saved to it.
  * Perform predictions on your test data with the loaded model and confirm they match the original model predictions from 4.

In [95]:
from pathlib import Path

# 1. Create models directory
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Create model save path
MODEL_NAME = "01_pytorch_workflow_model_exercise.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. Save the model state dict
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model.state_dict(), f=MODEL_SAVE_PATH)

In [97]:
# Create new instance of model and load saved state dict (make sure to put it on the target device)
loaded_model = LinearRegressionModel()
loaded_model.load_state_dict(torch.load(MODEL_SAVE_PATH))

loaded_model.state_dict()

In [104]:
# Make predictions with loaded model and compare them to the previous
loaded_device_type = next(loaded_model.parameters()).device
model.to(loaded_device_type)
X_test = X_test.to(loaded_device_type)

loaded_model.eval()
model.eval()
with torch.inference_mode():
  test_pred_original = model(X_test)
  test_pred_loaded = loaded_model(X_test)

(test_pred_loaded == test_pred_original).mT, test_pred_loaded.mT, test_pred_original.mT